### DEMO

In [2]:
import os
import torch
print(torch.cuda.is_available())

True


In [5]:
import torch
from gpt_model import GPT

if __name__ == '__main__':
    chooseEpoch = 12 # 
    modelPath = 'models/GPT2_epoch'+str(chooseEpoch)+'.pt'

    device = torch.device('cuda')
    model = GPT().to(device)
    model.load_state_dict(torch.load(modelPath))
    model.eval()
    
    #初始输入是空，每次加上后面的对话信息
    sentence = ''
    while True:
        temp_sentence = input("我:")
        sentence += (temp_sentence + '\t')
        if len(sentence) > 200:
            #由于该模型输入最大长度为300，避免长度超出限制长度过长需要进行裁剪
            t_index = sentence.find('\t')
            sentence = sentence[t_index + 1:]
        print("我:", temp_sentence)
        print("机器人:", model.answer(sentence))


我: 你是谁
机器人: 我是你的小可爱
我: 你猜猜我是谁
机器人: 你猜我猜不猜
我: 你猜猜我是谁
机器人: 你猜我猜不猜
我: 
机器人: 你猜我猜不猜
我: 
机器人: 你猜我猜不猜
我: 
机器人: 你猜我猜不猜


KeyboardInterrupt: Interrupted by user

## EVALUATION

In [15]:
import json
import torch
import time
from tqdm import tqdm
from nltk.tokenize import wordpunct_tokenize
from nltk.translate.bleu_score import corpus_bleu

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def getQuesGoldAns(validQAPath):
    Questions = []
    GoldenAnswers = []
    with open(validQAPath, 'r', encoding='utf-8') as src_file:
        for line in src_file:
            QA = line.strip().split('\t')
            Questions.append(QA[0])

            ga = QA[1]
            ga = list(ga)
            ga = " ".join(ga)
            ga = [wordpunct_tokenize(ga)]
            GoldenAnswers.append(ga)

    return Questions,GoldenAnswers

def GPT_BLUE_Evaluation(GPTModelPath, QAPath):
    device = torch.device('cuda')
    model = GPT().to(device)
    model.load_state_dict(torch.load(GPTModelPath))
    model.eval()

    Questions,GoldenAnswers = getQuesGoldAns(QAPath)
    ModelAnswers = []
    for Q in tqdm(Questions):
        A = model.answer(Q+'\t').strip()
        A = list(A)
        A = " ".join(A)
        A = wordpunct_tokenize(A)
        ModelAnswers.append(A)
    
    # print(GoldenAnswers[:5])
    # print(ModelAnswers[:5])

    bleu_score = corpus_bleu(GoldenAnswers, ModelAnswers) # 接下来要使这个东西连续三次不上升就跳出循环

    return bleu_score

if __name__ == '__main__':
    evaEpochList = [6]
    for e in evaEpochList:
        chooseEpoch = e # 
        modelPath = 'models/GPT2_epoch'+str(chooseEpoch)+'.pt'
        validQAPath = 'dataset/QA_valid_100.txt'

        device = torch.device('cuda')
        model = GPT().to(device)
        model.load_state_dict(torch.load(modelPath))
        model.eval()

        # evaluation
        print("Epoch %d Evaluation:"%(chooseEpoch), end='')
        start_time = time.time()
        Questions,GoldenAnswers = getQuesGoldAns(validQAPath)
        ModelAnswers = []
        for Q in tqdm(Questions):
            A = model.answer(Q+'\t').strip()
            A = list(A)
            A = " ".join(A)
            A = wordpunct_tokenize(A)
            ModelAnswers.append(A)
        
        print(GoldenAnswers[:5])
        print(ModelAnswers[:5])

        bleu_score = corpus_bleu(GoldenAnswers, ModelAnswers) # 接下来要使这个东西连续三次不上升就跳出循环
        end_time = time.time()
        eva_mins, eva_secs = epoch_time(start_time, end_time)
        print(f' | Time: {eva_mins}m {eva_secs}s')
        print("\tBLEUScore= %f"%(bleu_score))

Epoch 6 Evaluation:

100%|██████████| 100/100 [00:17<00:00,  5.84it/s]

[[['那', '个', '饭', '凉', '了', '吧', '唧', '的', '怎', '么', '吃', '啊', '摔']], [['看', '了', '下', '全', '文', '，', '那', '女', '的', '考', '试', '当', '天', '就', '表', '明', '身', '体', '不', '舒', '服', '了', '，', '考', '试', '不', '是', '她', '预', '约', '是', '教', '练', '自', '己', '安', '排', '的', '，', '教', '练', '还', '让', '她', '考', '试', '不', '就', '是', '教', '练', '的', '错', '吗', '？', '而', '且', '她', '住', '院', '花', '了', '3', '1', '万', '，', '赔', '3', '0', '万', '不', '过', '分', '吧']], [['更', '改', '要', '提', '前', '3', '天', '，', '当', '天', '不', '上', '场', '视', '为', '放', '弃', '考', '试', '，', '又', '要', '重', '新', '报', '名', '重', '新', '交', '钱', '，', '估', '计', '她', '也', '不', '肯', '的', '吧', '？']], [['好', '好', '好', '，', '偶', '遇', '我', '大', '闺', '蜜']], [['嗯', '，', '私', '聊', '吧', '，', '拜', '拜']]]
[['我', '也', '是', '，', '我', '也', '是', '，', '我', '们', '这', '边', '的', '虾', '米', '都', '是', '，', '我', '们', '这', '边', '的', '虾', '米', '都', '是', '，', '我', '们', '这', '边', '的', '虾', '米', '都', '是', '，', '我', '们', '这', '边', '的', '虾', '米', '都', '是', '，', '我', '们', '这


/home/ubuntu-user/.local/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


## Show Model Answers in QAData

In [3]:
import json
import torch
import time
from tqdm import tqdm
from nltk.tokenize import wordpunct_tokenize
from nltk.translate.bleu_score import corpus_bleu
import torch
from gpt_model import GPT

def getQuesGoldAns(validQAPath):
    Questions = []
    GoldenAnswers = []
    with open(validQAPath, 'r', encoding='utf-8') as src_file:
        for line in src_file:
            QA = line.strip().split('\t')
            Questions.append(QA[0])
            GoldenAnswers.append(QA[1])

    return Questions,GoldenAnswers


if __name__ == '__main__':
    evaEpochList = [12]
    for e in evaEpochList:
        chooseEpoch = e # 
        modelPath = 'models/GPT2_epoch'+str(chooseEpoch)+'.pt'
        validQAPath = 'dataset/QA_valid_100.txt'

        device = torch.device('cuda')
        model = GPT().to(device)
        model.load_state_dict(torch.load(modelPath))
        model.eval()

        # evaluation
        print("Epoch %d Ans show:"%(chooseEpoch), end='')
        Questions,GoldenAnswers = getQuesGoldAns(validQAPath)
        ModelAnswers = []
        for Q in tqdm(Questions):
            A = model.answer(Q+'\t').strip()
            ModelAnswers.append(A)
        
        for q,a in zip(Questions, ModelAnswers):
            print("Ask: %s\nModelAns: %s"%(q,a))

Epoch 12 Ans show:

100%|██████████| 100/100 [00:19<00:00,  5.03it/s]

Ask: 啊我好爱虾仁蛋黄酱金枪鱼蛋黄酱
ModelAns: 我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我
Ask: 考试撞墙关驾校屁事？你怎么不顺便把考场施工单位也告了？
ModelAns: 你这是在黑河南的节奏吗？
Ask: 看了下全文，那女的考试当天就表明身体不舒服了，考试不是她预约是教练自己安排的，教练还让她考试不就是教练的错吗？而且她住院花了31万，赔30万不过分吧
ModelAns: 我们这边的考试不是她的错，她是她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的惩罚她的惩罚她的惩罚她的惩罚她的惩罚她的惩罚她的惩罚她的惩罚她的
Ask: 我先去穿衣服，准备走了
ModelAns: 我也想去
Ask: 好好好，偶遇我大闺蜜
ModelAns: 我也是，我们一起去的
Ask: 期待小猎豹的表现
ModelAns: 我也是
Ask: 为了你们新年有惊喜，我也是用心良苦了
ModelAns: 新年快乐
Ask: 世界上最远的距离，不是爱，不是恨，而是熟悉的人，渐渐变得陌生。
ModelAns: 是的，我们都在，我们在的是不是爱，但是我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我
Ask: 生日快乐！年年18
ModelAns: 谢谢
Ask: 你是18那我17好了
ModelAns: 我17
Ask: 青青妹！！！好久不见！别来无恙！！假期也没见你呜呜呜！我不想你，谁想你谁小狗呜呜呜呜呜呜呜！汪！
ModelAns: 我也是！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
Ask: 文艺小青年
ModelAns: 你是不是傻
Ask: 你知道这期录的什么时候播不
ModelAns: 不知道
Ask: 不知道
ModelAns: 我也是
Ask: 好吧，我去建房子了
ModelAns: 你去

## EVALUATION FOR BERT SCORE

In [7]:
# sentence-lv bert_score sample
from bert_score import score as bert_score

ModelAnswers = ['我们都是星期天交的', '我们都是星期一交的']
GoldenAnswers = ['我们都是星期天交的', '我们都是星期天交的']

P, R, F1 = bert_score(ModelAnswers, GoldenAnswers, lang='zh')
avg_bert_score = F1.mean().item()
print(avg_bert_score)

0.9871877431869507


In [1]:
import json
import torch
import time
from tqdm import tqdm
from bert_score import score as bert_score
from gpt_model import GPT

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def getQuesGoldAns(validQAPath):
    Questions = []
    GoldenAnswers = []
    with open(validQAPath, 'r', encoding='utf-8') as src_file:
        for line in src_file:
            QA = line.strip().split('\t')
            Questions.append(QA[0])
            GoldenAnswers.append(QA[1])

    return Questions,GoldenAnswers



if __name__ == '__main__':
    evaEpochList = list(range(28,29))
    
    best_bert_score = -1
    prev_bert_score = -1
    no_improvement_count = 0
    all_bert_score_evaluation_info = []

    for e in evaEpochList:
        chooseEpoch = e # 
        modelPath = 'models/GPT2_epoch'+str(chooseEpoch)+'.pt'
        validQAPath = 'dataset/QA_test.txt'

        device = torch.device('cuda')
        model = GPT().to(device)
        model.load_state_dict(torch.load(modelPath))
        model.eval()

        # evaluation
        print("Epoch %d Evaluation:"%(chooseEpoch), end='')
        start_time = time.time()
        Questions,GoldenAnswers = getQuesGoldAns(validQAPath) 
        ModelAnswers = []
        for Q in tqdm(Questions):
            A = model.answer(Q+'\t').strip()
            ModelAnswers.append(A)
        
        # print(GoldenAnswers[:5])
        # print(ModelAnswers[:5])

        # 接下来要使这个东西连续三次不上升就跳出循环
        P, R, F1 = bert_score(ModelAnswers, GoldenAnswers, lang='zh')
        avg_bert_score = F1.mean().item()
        end_time = time.time()
        eva_mins, eva_secs = epoch_time(start_time, end_time)

        if best_bert_score <= avg_bert_score:
            best_bert_score = avg_bert_score

        if prev_bert_score > avg_bert_score:
            no_improvement_count += 1
        else:
            no_improvement_count = 0 

        bert_score_evaluation_info = {
            'epoch': chooseEpoch,
            'bert_score': avg_bert_score,
            'best_bert_score': best_bert_score,
            'no_improvement_count': no_improvement_count
        }

        all_bert_score_evaluation_info.append(bert_score_evaluation_info)

        with open('models/bert_score_evaluation_info.json', 'w') as json_file:
            json.dump(all_bert_score_evaluation_info, json_file)

        prev_bert_score = avg_bert_score

        print(f' | Time: {eva_mins}m {eva_secs}s')
        print("avg_bert_score= %f | best_bert_score= %f | no_improvement_count = %d"%(avg_bert_score,best_bert_score,no_improvement_count))



Epoch 28 Evaluation:

 12%|█▏        | 2301/19008 [01:36<11:41, 23.82it/s]


KeyboardInterrupt: 